In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec

# mne library to analyse EEG
import mne
from mne import Epochs, pick_types
from mne.channels import make_standard_montage
from mne.datasets import eegbci
from mne.decoding import CSP
from mne.io import concatenate_raws, read_raw_edf
mne.set_log_level('error') # Avoid long log

In [ ]:
# Get the path to the data
def file_path(participant, run):
    return f'files/S{participant:03}/S{participant:03}R{run:02}.edf'

def load_raws(participant: list, runs: list, preload = True, path: str = 'files/'):
    raws = concatenate_raws([read_raw_edf(path + 'S{participant:03}/S{participant:03}R{run:02}.edf', preload = preload) for participant in range(1,11) for run in runs])
    return raws

In [ ]:
# Create a list with each tipe of experimental run
openeye_runs = [1]
closedeye_runs = [2]
fists_runs = [3, 7, 11]
imaginefists_runs = [4, 8, 12]
fistsfeet_runs = [5, 9, 13]
imaginefistsfeet_run = [6, 10, 14]

def load_raws(participant: list, runs: list, preload = True, path: str = 'files/'):
    raws = [read_raw_edf(path + 'S{participant:03}/S{participant:03}R{run:02}.edf', preload = preload) for participant in range(1,11) for run in runs]
    return raws

# Function to load the data and set the montage
def load_and_set_montage(raws: list, montage_type: str = "Standard_1020"):

    # f no other choise is made we try the "Standard_1020" montage
    montage = make_standard_montage(montage_type)

    # Dictionary with the structure old_name : correct_cases_name. To respect the upper and lower cases of the standard notation for the electrde's position.
    replacement = {
        'Fc': 'FC',
        'Cp': 'CP',
        'Af': 'AF',
        'Ft': 'FT',
        'Tp': 'TP',
        'Po': 'PO'   
    }

    
    # new_name is the dictionary to use to cange the name of the electrode's positions to respect the usual sandard notataions.
    # First get rid of the excessive "." 
    new_names = {
    name : name.replace(".", "") for name in raw.info['ch_names']
    }   

    # Change the lower and upper case of the electrode's names
    for key in new_names.keys():
        for old_string, new_string in replacement.items():
            new_names[key] = new_names[key].replace(old_string, new_string)

    # Rename channels and set the montage
    for raw in raws:
        raw.rename_channels(new_names)
        raw.set_montage(montage)

    return raws


# Function to filter the data
def filter(raws : list, low_cut: float = 0.1, high_cut: float = 30, preload = False):

    if preload == True:
        raws_filt = [raw.copy().filter(low_cut, high_cut) for raw in raws]
    else:
        raws_filt = [raw.copy().load_data().filter(low_cut, high_cut) for raw in raws]
    
    return raws_filt


def create_epochs(raws_filt, low_cut: float = 0.1, high_cut: float = 30, time_step: float = 1, baseline = None, event_id = {'rest': 1, 'left_fist': 2, 'right_fist': 3}):
    epochs = [
        Epochs(raw_filt, mne.events_from_annotations(raw_filt)[0], event_id, tmin=tmin, tmax=tmax, baseline= baseline, preload= True)
        for raw_filt in raws_filt
    ]
    return epochs



# Creating a raw data to use to compute the ICA
raw_ica = raw.copy().filter(ica_low_cut, high_cut)

# Creating the epochs for the artifact detection and the ICA
time_step = 1 # Coerent with the time-span of the most common artifact (heart, eye blink)
events_ica = mne.make_fixed_length_events(raw_ica, duration=time_step)
epochs_ica = mne.Epochs(raw_ica, events_ica,
                        tmin=0.0, tmax=time_step,
                        baseline=None,
                        preload=True)










from autoreject import AutoReject


# Filter settings for ICA
ica_low_cut = 1
high_cut = 30

# Creating a raw data to use to compute the ICA
raw_ica = raw.copy().filter(ica_low_cut, high_cut)

# Creating the epochs for the artifact detection and the ICA
time_step = 1 # Coerent with the time-span of the most common artifact (heart, eye blink)
events_ica = mne.make_fixed_length_events(raw_ica, duration=time_step)
epochs_ica = mne.Epochs(raw_ica, events_ica,
                        tmin=0.0, tmax=time_step,
                        baseline=None,
                        preload=True)

# We use the autoreject function to clean clean the data before applying ICA, indeed it automatically 
# determine a threshold to use to find sections of the data that are excessively noisy. 
# This can occur, for example, if a participant scratches their head during a recording, or sneezes, or for various other reasons.

# Instancing the autoreject object
ar = AutoReject(
                random_state=42,
                picks=mne.pick_types(epochs_ica.info, 
                                     eeg=True,
                                     eog=False
                                    ),
                n_jobs=-1, 
                verbose=False
                )

ar.fit(epochs_ica)

reject_log = ar.get_reject_log(epochs_ica)
reject_log.plot('horizontal', aspect='auto');

# Filter settings for ICA
ica_low_cut = 1
high_cut = 30

# Creating a raw data to use to compute the ICA
raw_ica = raw.copy().filter(ica_low_cut, high_cut)

# Creating the epochs for the artifact detection and the ICA
time_step = 1 # Coerent with the time-span of the most common artifact (heart, eye blink)
events_ica = mne.make_fixed_length_events(raw_ica, duration=time_step)
epochs_ica = mne.Epochs(raw_ica, events_ica,
                        tmin=0.0, tmax=time_step,
                        baseline=None,
                        preload=True)

# We use the autoreject function to clean clean the data before applying ICA, indeed it automatically 
# determine a threshold to use to find sections of the data that are excessively noisy. 
# This can occur, for example, if a participant scratches their head during a recording, or sneezes, or for various other reasons.

# Instancing the autoreject object
ar = AutoReject(
                random_state=42,
                picks=mne.pick_types(epochs_ica.info, 
                                     eeg=True,
                                     eog=False
                                    ),
                n_jobs=-1, 
                verbose=False
                )

ar.fit(epochs_ica)
